In [1]:
#環境変数を読み込む
from dotenv import load_dotenv
load_dotenv()

True

In [3]:
# ライブラリのimport
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.agents import ZeroShotAgent, Tool, AgentExecutor
from langchain.memory import ConversationBufferMemory
from langchain import LLMChain
from langchain.utilities import GoogleSearchAPIWrapper
import re

In [4]:
#推論で用いるLLMを用意する(ここはgeminiでもchatgptでもllamaでもok.もちろんpromptチューニングは必要)
llm = ChatGoogleGenerativeAI(model="gemini-pro")

In [5]:
# langchainを使ったToolを使って自分でコンテキスト情報を追加し、最終回答導くAgentを構築する
def remove_empty_lines(text):
    # 正規表現を使用して改行と改行の間に文字がない行を削除
    cleaned_text = re.sub(r'\n\s*\n', '\n', text)
    return cleaned_text

def search_and_summarise(query):
  order = re.split(r'[:;,の.、 　]', query)
  result = search.run(' '.join(order[::-1]))
  res = '\n'.join([info.strip() for info in result.split('...') if info.find('www')==-1 and info.find('@')==-1 and info.find('http')==-1])

  return remove_empty_lines(f"""search result
                            {res}""")

memory = ConversationBufferMemory(memory_key="chat_history")

# ツールを作成
search = GoogleSearchAPIWrapper()
tools = [
    Tool(
        name = "検索",
        func=search_and_summarise,
        description="It is useful for answering questions about current events. The input format for this tool must always follow the format of a comma-separated string of topics and things you want to know about the topic."
    ),
    
]

prefix = """あなたはユーザの日常生活を補助するアシスタントです, 質問は可能な限り正確に日本語で回答してください. あなたはツールの指示に必ず従わなければいけません:"""
suffix = """始めましょう!質問に対してステップバイステップで調査し、必ずフォーマットに従って回答します。

{chat_history}
Question: {input}
{agent_scratchpad}"""

prompt = ZeroShotAgent.create_prompt(
    tools,
    prefix=prefix,
    suffix=suffix,
    input_variables=["input", "chat_history", "agent_scratchpad"]
)
llm_chain = LLMChain(llm=llm, prompt=prompt)

# エージェントを作成
agent = ZeroShotAgent(llm_chain=llm_chain, tools=tools, verbose=True, max_iterations=5)

# Memoryを含むAgentExecutorチェーンを作成
# エージェントはMemoryを編集して良いので、そのまま渡す
agent_chain = AgentExecutor.from_agent_and_tools(agent=agent, tools=tools, verbose=True, memory=memory)


ValidationError: 1 validation error for GoogleSearchAPIWrapper
__root__
  Did not find google_cse_id, please add an environment variable `GOOGLE_CSE_ID` which contains it, or pass  `google_cse_id` as a named parameter. (type=value_error)

In [6]:
# Agentを呼び出すための関数を定義する

# 質問に答えてもらう関数を定義する
def answer_question(question):
  # 質問にこたえてもらう
  result = agent_chain.run(question)
  return result

# 過去の会話履歴を削除する関数を定義する
def clear_old_conversation():
  memory.clear()


In [26]:
# 動作確認
print(answer_question('聖徳太子についておしえて'))



> Entering new AgentExecutor chain...
Thought: 聖徳太子について知りたいので 検索してみよう
Action: 検索
Action Input: 聖徳太子
Observation: search result
                            聖徳太子（しょうとくたいし、旧字体: 聖󠄁德太子）は、飛鳥時代の皇族・政治家。用明天皇の第二皇子で、母は欽明天皇の皇女・穴穂部間人皇女。 お太子さまは『法華経』『勝鬘経』『維摩経』を講義するなど仏教に通じ、冠位十二階や憲法十七条を制定されたり、社会福祉事業として四箇院の制を実践されました。 片岡山
「東京ドームシティ」内に、東京ドーム×吉本興業グループによる、演劇や演芸等、さまざまなエンターテインメントの制作者やパフォーマー待望の劇場が誕生！ 誰もがその名を知る、日本古代史最大級の偉人「聖徳太子」。かつて旧1万円札の代名詞でもあり、「一度に10人の話を同時に聞き分ける」など、その生涯における超人的
推古天皇と聖徳太子（厩戸皇子）との複雑な血縁関係. 女性天皇として知られる推古天皇（すいこてんのう）は、554年（欽明天皇15年）生まれ。父は第29代欽明天皇（きんめい
May 25, 2016
用明天皇の皇子として誕生し、母は蘇我稲目（そがのいなめ）の孫にあたる穴穂部間人皇女（あなほべのはしひとのひめみこ）です。19歳で推古天皇の摂政
Feb 3, 2021
寺伝では、聖徳太子が蘇我馬子の支援を得て建立された、と言われています。 ( ) ( ) 「中の太子」とも呼ばれ、太子町の「叡福寺」（上の太子）、八尾市の
歴史にドキリ
お使いのブラウザではビデオの再生はサポートしていません。
6世紀末の日本。大和朝廷で摂政の役職にあった聖徳太子は、豪族・蘇我馬子とともに、天皇
推古30年(622年)の旧暦2月22日(太陽暦4月11日)、太子は病で薨去されました。御年49歳。 日月は輝きを失い、人々は嘆き悲しんで、太子と前日に亡くなられた妃・膳部大郎女と
Aug 29, 2019
道後温泉 ( どうごおんせん ) 別館 ( べっかん ) 飛鳥 ( あすか ) 乃 ( の ) 湯泉 ( ゆ ) の 中庭 ( なかにわ ) に、 聖徳太子 ( しょうとくたいし ) の

Thou

In [27]:
clear_old_conversation()
print(answer_question('ARISEanalyticsってどんな会社?'))



> Entering new AgentExecutor chain...
Thought: ARISEanalyticsについての情報は検索する必要がある
Action: 検索
Action Input: ARISEanalytics
Observation: search result
                            ARISE is a family of integrated toolkits used to build a system-level weapon simulation – or “digital twin” – tool. 株式会社ARISE analytics（アライズ アナリティクス）は、国内最大規模のデータと最先端のデータ処理・分析技術を有するデータサイエンティスト集団。 Oct 5, 2023
ARISE allows our engineers to rethink how we handle, ingest and save data while minimizing resource load on our IT infrastructures. With a
ARISE Analytics provides data analysis, algorithm development, and analytic services such as DMP, AI, and IoT solution introduction support. Arise Agency | 23 followers on LinkedIn. Stay ahead of your competition! | Stay ahead of your competition World leading reliable Prospect Market Research
ARISE analytics（アライズ アナリティクス）は、KDDIとアクセンチュアのジョイントベンチャー。約300名のデータサイエンティストが在籍し、データアナリティクスと
CU Rise Analytics | 1988 followers on LinkedIn. Helping Credit Unions Rise with Analytics | CU

In [28]:
print(answer_question('どこにある?'))



> Entering new AgentExecutor chain...
Thought: まずは株式会社ARISE analyticsのウェブサイトを見つけることから始めましょう。
Action: 検索
Action Input: サイト名:株式会社ARISE analytics
Observation: search result
                            株式会社ARISE analytics（アライズ アナリティクス）は、国内最大規模のデータと最先端のデータ処理・分析技術を有するデータサイエンティスト集団。 Apr 10, 2023
より大きな取り組みにするために、KDDIとアクセンチュアが興したジョイントベンチャーが当社です。KDDIから5名ほど、アクセンチュアからは40名ほどが参画
現在350名を越えるデータサイエンティストが活躍しており、データ分析専業の企業では国内最大規模を誇ります! DX時代の到来により、最先端技術を正しく活用しながらビッグ
必須会社名. 任意部署. 任意役職. 必須お名前. 必須メールアドレス. 必須電話番号. 必須メッセージ. 任意初めて弊社を知ったきっかけは何ですか.
サイト利用者の商品購入や閲覧の履歴好みを予測して、利用者に欲しそうな物を提案
弊社には、350名を超えるデータサイエンティストが在籍しています。各チームが
株式会社ARISE analytics. データ分析、アルゴリズム開発、及びDMP、AI、IoTソリューション導入支援などのアナリティクスサービスの提供. 求人
Nov 16, 2022
株式会社ARISE analyticsで実際「データサイエンティスト」として働く社員さんが感じる働きやすさ・働きがいについてインタビューした記事になり
Mar 22, 2023
株式会社ARISE analyticsの法人番号は7011001115102です。最新の登記情報では、2017年3月に新規設立または法人番号登録がされました。 -名
Nov 25, 2019
□本プログラムの概要当社は2017年の設立以降、300社3000名以上へデータサイエンティスト育成を目的としたオンライン学習プログラムを提供してまいりまし
株式会社ARISE analyticsの企業情報。「【KDDI×アクセ

In [29]:
print(answer_question('最寄駅はどこ?'))



> Entering new AgentExecutor chain...
Thought: 住所の場所から、最寄り駅を調べる。
Action: 検索
Action Input: 東京都渋谷区渋谷2丁目21番1号
Observation: search result
                            住所: 〒150-8510 東京都渋谷区渋谷2-21-1; アクセス: JR線・京王井の頭線「渋谷駅」と2階連絡通路で直結 東京メトロ銀座線「渋谷駅」と1階で直結 名称, 株式会社アダストリア. Adastria Co., Ltd. 本部住所, 〒150-8510 東京都渋谷区渋谷2丁目21番1号 渋谷ヒカリエ 27F. 設立, 1953年10月22日. 資本金, 2,660百万円. 渋谷ヒカリエ. 〒150-8510 東京都渋谷区渋谷2-21-1. JR線・京王井の頭線「渋谷駅」と2階連絡通路で直結; 東京メトロ銀座線「渋谷駅」と1階で直結; 東急東横線・田園都市線
〒150-8510 東京都渋谷区渋谷2-21-1 渋谷ヒカリエ33階. 月～金、9:00am～6:00pm（営業時間外：土日祝日）. 渋谷ヒカリエ. 渋谷駅に直結しています。2階からはJR・京王
アクセス. 150-0002 東京都渋谷区渋谷2-21-1 渋谷ヒカリエ8階 8 ⁄（ハチ） Google map
⁄ CUBE 1, 2, 3; ⁄ ART GALLERY ⁄ TOMIO KOYAMA GALLERY; ⁄ d47 MUSEUM
住所. 〒150-8510 東京都渋谷区渋谷2-21-1 渋谷ヒカリエ11階. 公共交通機関からのアクセス. JR線、京王井の頭線「渋谷駅」と2階連絡通路で直結東京メトロ銀座線「渋谷
2023年12月21日（木）. JPDA 創作展「水 塩 米 酒」の開催を記念して、特別企画講演を
〒150-8510 東京都渋谷区渋谷2-21-1 渋谷ヒカリエ8階 お問い合わせ. FOLLOW US
ACCESS アクセス 〒150-8510 東京都渋谷区渋谷2丁目21−1渋谷ヒカリエ 32F [Google.
：11時～21時. ※一部営業時間が異なるショップがございます。 ▷ 詳しくはこちらをご覧ください。 〒150-8509 東京都渋谷区渋谷2-21-

In [30]:
print(answer_question('千葉駅からどうやったらいける?'))



> Entering new AgentExecutor chain...
Thought: 最初に渋谷駅までの行き方を検索する
Action: 検索
Action Input: 千葉駅から渋谷駅まで行く方法
Observation: search result
                            千葉から渋谷への乗り換え案内です。電車のほかに新幹線、飛行機、バス、フェリーを使用するルートもご案内。IC運賃、定期券料金、時刻表、運行状況、駅周辺の地図も
千葉から渋谷までの乗換案内。電車を使った経路を比較。時刻、乗換回数、所要時間、運賃・料金を案内。 △ … 前後の時刻表から計算した推定時刻です。 ( ) … 徒歩/車を使用した場合の時刻です。 到着駅を指定した直通時刻
千葉から渋谷までの電車の運賃・料金を案内。ICときっぷ、片道・往復で表示。交通費の精算や旅費の計算に便利。 3 · JR山手線内回り・渋谷・品川方面 · [発] 2番線 / [着] 2番線 · 2駅. 05:52: 原宿. Feb 13, 2009
もしチェックされない場合は、りんかい線の定期券を会社が認めてくれれば、 途中下車は無理だとしても、 かなり複数の行き方を選択できるようになります
Apr 12, 2011
やはり、秋葉原～（総武線）～代々木～（山手線内回り）～渋谷だと思います。 May 27, 2023
表参道に用があって千葉駅から総武線快速で行こうと思ったのですが、「成田エクスプレス新宿・大船行き」がまもなく千葉駅に到着すると表示されていた
高速バス・深夜バスのよくある質問 · 兵庫にある主要なバスターミナルはどこですか？ · 兵庫発の高速バス運休情報が知りたいです。 · JR新宿駅からバスタ新宿への行き方を教え
運賃とお支払い方法. 運賃. 渋谷マークシティ～袖ケ浦バスターミナル. 片道. 大人：1,500円小児：750円. 渋谷マークシティ～木更津駅西口. 片道. 大人：1,600円小児：800円.
Thought:千葉駅から渋谷駅までの乗り換え案内を検索する
Action: 検索
Action Input: 千葉駅から渋谷駅まで行く方法
Observation: search result
                            千葉から渋谷

In [31]:
print(answer_question('どのくらいかかる?'))



> Entering new AgentExecutor chain...
Thought: いまの私の知識では、所要時間は分かりません。
Action: 検索
Action Input: 秋葉原駅から渋谷駅までの所要時間
Observation: search result
                            秋葉原から渋谷への乗り換え案内です。電車のほかに新幹線、飛行機、バス、フェリーを使用するルートもご案内。IC運賃、定期券料金、時刻表、運行状況、駅周辺の地図も
△ … 前後の時刻表から計算した推定時刻です。 ( ) … 徒歩/車を使用した場合の時刻です。 到着駅を指定した直通時刻
秋葉原から渋谷までの電車の運賃・料金を案内。ICときっぷ、片道・往復で表示。交通費の精算や旅費の計算に便利。 stainfo駅情報 · stadia時刻表 · myマイページ. 秋葉原→渋谷. ほかに候補があります. 駅・条件を変更. 発: 秋葉原駅. 着. 渋谷駅, 高座渋谷駅. 1本前; 2023年12月16日(土)
秋葉原から渋谷までの最終電車を案内。電車を使った経路を比較。時刻、乗換回数、所要時間、運賃・料金を案内。 Oct 12, 2011
外回りの方が早いです。 外回り（東京、品川経由）が約25分内回り（上野、新宿経由）が約35分但し最速は総武線・中央線各駅停車（新宿方面行き）で
Aug 9, 2008
10分程度所要時間が違います。 平日日中で内回り26分、外回り34分かかり
夜に帰る際、表参道駅から渋谷へ向かおうとしたら、路線案内には表参道か. 04渋谷・原宿・六本木東京タワー方面; 05新宿・池袋方面. 01日本橋・銀座・皇居方面. ※()内、水天宮前駅からの目安所要時間. ①人形町. 人形町駅（徒歩約7分）. 東京メトロ
Aug 31, 2017
東急東横線は、終点の横浜駅から渋谷駅まで快速急行で乗車30分以内、横浜駅の隣の各駅停車駅である反町駅からでも乗車40分以内とアクセスがよく便利です。 秋葉原駅 ➔ 笠間・益子 · 大子・大宮・太田線. 上り 大子・大宮・常陸太田 ➔ 那珂
PC・スマートフォンから時刻表検索 · 採用情報 · 巡る・食べる 茨城Maas · 茨城交通

Thought: 複数のサイトを調べたところ、秋葉原駅から渋

In [35]:
clear_old_conversation()
print(answer_question('協調フィルタリングをtensorflowで実装してほしい'))



> Entering new AgentExecutor chain...
Thought: Tensorflowはオンラインで利用できるリソースが豊富で、協調フィルタリングの実装に関するチュートリアルやガイダンスが数多くあります。これらのリソースを利用して、協調フィルタリングをTensorflowで実装する方法を調べてみます。
Action: 検索
Action Input: Tensorflow、協調フィルタリングの実装の仕方
Observation: search result
                            Dec 18, 2018
細かい解説は省きますが、ニューラル協調フィルタリングの図をTensorFlow/Kerasを使ってコードに落としてみたものが下記になります。Embedding Layerの
Nov 17, 2016
今回は，Pythonで協調フィルタリング（の原型）を実装してみた．初めにNumpy+Scipy(Optimize)で動作を確認し，次にTensorFlowを用いた実装を確認した． Sep 21, 2018
TensorFlowのCPU / GPUバージョンでそれぞれパフォーマンスを評価します。 協調フィルタリング. ユーザーからもらった別々のデータが協力して一つの推薦
Sep 18, 2022
レコメンドシステムにはさまざまなアルゴリズムが存在します。 コンテンツベース; 協調フィルタリング; Matrix Factorization; Factorization Machines
Aug 25, 2017
第2回では、協調フィルタリングの実装を行いました。本連載では今後各種手法を実装しながら比較していく予定ですが、その前にレコメンドの評価について
Mar 30, 2018
協調フィルタリングを行うNeural Matrix Factorization (NeuMF)を実装しました．
TensorFlow Wide & Deep Learning Tutorial | TensorFlow. The figure
May 23, 2017
そして今回、ようやく訪れた連載第2回は、ユーザベース協調フィルタリングを実装し、実際のデータに対して適用してみたいと思います。 □今回やること. Oct 11, 201